# Student Alcohol Consumption
Link: https://www.kaggle.com/datasets/uciml/student-alcohol-consumption

In [14]:
import numpy as np
import pandas as pd

# There is two data sets, one for math grades, the other for portuguese
math_df = pd.read_csv('Datasets/Student Alcohol Consumption/student-mat.csv')
por_df =  pd.read_csv('Datasets/Student Alcohol Consumption/student-por.csv')

# print(np.shape(math_df))        # Shape [395, 33]
# print(np.shape(portuguese_df))  # Shape [649, 33]

The main goal here is to predict the student's grades (both math and portuguese) using relevant features.

Need to parse the data set so the values can be used (i.e. yes/no should be changed to 1/0).

In [15]:
# Changing yes/no features to 1/0
yes_no_cols = [col for col in math_df.columns if set(math_df[col].unique()) == {'yes', 'no'}]
for col in yes_no_cols:
  math_df[col] = np.where(math_df[col] == 'yes', 1, 0)

# print(math_df.iloc[0, :])
math_features = math_df.iloc[:, :30]
math_labels = math_df.iloc[:, 30:33]
por_features = por_df.iloc[:, :30]
por_labels = por_df.iloc[:, 30:33]

In [16]:
# Given a data frame, generate the training/validation set's features and labels
def generateRandom(df, features, labels):
  perm_idx = np.random.permutation(df.shape[0])  # Shuffling entries around
  vali_num = int(df.shape[0] * 0.2)              # Getting 20% of the data set
  vali_idx = perm_idx[:vali_num]
  train_idx = perm_idx[vali_num:]
  train_features = features.iloc[train_idx, :]
  train_labels = labels.iloc[train_idx]
  vali_features = features.iloc[vali_idx, :]
  vali_labels = labels.iloc[vali_idx]
  return train_features, train_labels, vali_features, vali_labels

math_train_features, math_train_labels, math_vali_features, math_vali_labels = generateRandom(math_df, math_features, math_labels)
por_train_features, por_train_labels, por_vali_features, por_vali_labels = generateRandom(por_df, por_features, por_labels)
# print(math_train_features.shape)
# print(math_vali_features.shape)
# print(por_train_features.shape)
# print(por_vali_features.shape)

In [17]:
# def KNN(train_features, train_labels, test_features, k):
    # vali_pred = []
    # for i in range(test_features.shape[0]):
    #     x = test_features[i, :]
    #     distances = np.sqrt(np.sum((x - train_features) ** 2, axis=1))
    #     topk_idx = np.argpartition(distances, k)[:k]
    #     topk_labels = list(train_labels[topk_idx])
    #     pred = max(topk_labels, key=topk_labels.count)
    #     vali_pred.append(pred)
    # return np.array(vali_pred)
    
# vali_pred = KNN(math_train_features, math_train_labels, math_vali_features, k=10)